In [6]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ORBSLAMResults, umeyama_alignment, kabsch_umeyama

results = ORBSLAMResults("~/msc/shared_data/orbslam-out-lest-umeyama-vabadusepst")

gps_trajectory = pd.DataFrame([(kf.gps.x, kf.gps.y, kf.gps.z) for kf in results.keyframes[1:]], columns=[ 'x', 'y', 'z'])
slam_trajectory = pd.DataFrame([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]], columns=[ 'x', 'y', 'z'])
R, t, c = umeyama_alignment(slam_trajectory.to_numpy().T, gps_trajectory.to_numpy().T,   True)
adjusted_slam_trajectory = pd.DataFrame(np.array([t + c * R @ p for p in slam_trajectory.to_numpy()]), columns=[ 'x', 'y', 'z'])
estimated_slam_trajectory = pd.DataFrame([(kf.x, kf.y, kf.z) for kf in results.gps_estimate[1:]], columns=[ 'x', 'y', 'z'])

fig = go.Figure(layout=dict(title='Actual vs. SLAM trajectory', height=800))
fig.add_trace(go.Scatter(x=gps_trajectory.x, y=gps_trajectory.y, mode='lines', name='GPS trajectory'))
fig.add_trace(go.Scatter(x=adjusted_slam_trajectory.x, y=adjusted_slam_trajectory.y, mode='lines', name='transformed SLAM trajectory'))
fig.add_trace(go.Scatter(x=estimated_slam_trajectory.x, y=estimated_slam_trajectory.y, mode='lines', name='estimated SLAM trajectory'))
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()


[[-0.98868165  0.00212519 -0.15001362]
 [ 0.15000281  0.03256588 -0.98814909]
 [ 0.00278532 -0.99946733 -0.03251607]]
[9243.00911897 9820.13317225   36.92023815]
68.26409742471482
